In [1]:
import pickle

import datetime
import pandas as pd

from data.dao import DataAccess, LabelGetter
from data.turk import TurkResults2Label

In [2]:
import os

In [6]:
folder = 'E:/Sept-March 2015/csv with offset/sept amt'
tweets = pd.DataFrame()
for file in os.listdir(folder):
    df = pd.read_csv(folder + '/' + file, engine='python')
    tweets = pd.concat([tweets, df])

In [7]:
tweets.shape

(682931, 7)

In [ ]:
import re

In [8]:
keywords = set('''drink drinker drinks drinking drank wine champgne alcohol alcoholics alcoholism beer beers bottle bottles
    pint pints cocktail cocktails bar brewery lounge pub liquor booze vodka tequila gin ciroc margarita margaritas shot shots
    ale whiskey lager tipsy drunk sober wasted pregame pregaming'''.split())

def prefilter(text):
    return bool(set(text.split()).intersection(keywords))

In [12]:
tweets.text.shape

(682931,)

In [13]:
filtered = tweets[tweets.text.apply(prefilter)]

In [16]:
filtered.sample(1000)

,Unnamed: 0,text,id,created_at,lat,lon,utc_offset
95402,23211,Teen drinking is very bad,648035279631085569,Sun Sep 27 07:23:45 +0000 2015,43.068737,-88.540000,NaN
55843,54590,so pour a shot in my glass and I'll forget for...,640984252180484096,Mon Sep 07 20:25:29 +0000 2015,30.144425,-88.473228,NaN
24112,92816,@CynthiaNewsome it depends on what day. Someti...,643511933669371908,Mon Sep 14 19:49:35 +0000 2015,38.868002,-94.845486,NaN
62895,155721,Helllooo new #wine bar discovery!! Let there b...,641067852146061312,Tue Sep 08 01:57:41 +0000 2015,32.539620,-117.282538,-25200.0
27258,109973,Kim engaged?? What that open bar like DOE?,646909652081381377,Thu Sep 24 04:50:54 +0000 2015,33.825123,-84.348581,-21600.0
9864,64208,"Ivory Ella's water bottles are sold out, and i...",643248685502066688,Mon Sep 14 02:23:32 +0000 2015,30.144425,-88.473228,NaN
70084,49661,@valsingerr lol ya I drink so I can love you,649274616389718016,Wed Sep 30 17:28:26 +0000 2015,41.117660,-81.392451,-10800.0
38161,14495,JB bought a drink that is supposedly a muscle ...,643652141966589952,Tue Sep 15 05:06:43 +0000 2015,27.386503,-82.459348,-18000.0
61057,16594,"@g0ldieex thank you very much, come out for dr...",644015026295209984,Wed Sep 16 05:08:42 +0000 2015,33.845960,-118.108568,-25200.0
72914,65104,#Mcm goes out to lfrancoeur96 the best beer po...,646104533127557120,Mon Sep 21 23:31:39 +0000 2015,44.961937,-76.353876,NaN


In [17]:
filtered['random_number'] = filtered.text.apply(hash)

C:\Users\Tom Work\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
filtered['_id'] = filtered.id
filtered.index = filtered['_id']

C:\Users\Tom Work\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
pickle.dump(filtered, open('pickles/amt-sept.p', 'wb'))

In [34]:
import os
def make_batch(pickled_df, size, output_dir):
    """
    pickled_df: unlabeled data with 'random_number' column
    size: # of items batch to be created
    output_dir: where the outputs are saved
    """
    # Load data
    X = pickle.load(open(pickled_df, 'rb'))
    
    # select size subset
    batch = X.sort_values(by='random_number').head(size)
    
    # process for turk
    batch["text"] = batch.text.str.encode("utf-8")
    date = str(datetime.date.today())
    time = datetime.datetime.now().time()
    key = batch.iloc[0].random_number
    
    # persist
    description = "/c_amt {} {}h{}m {}".format(date, time.hour, time.minute, key)
    if not os.path.exists(output_dir + description):
        os.makedirs(output_dir + description)
    batch[["_id", "text", "random_number"]].to_csv(output_dir + description +  description + '.csv')
    pickle.dump(batch, open(output_dir + description + description +".p", 'wb'))
    X = X.drop(batch.index)
    pickle.dump(X, open(pickled_df, 'wb'))

In [35]:
make_batch('pickles/amt-sept.p', 1000, 'amt')

In [40]:
make_batch('pickles/amt-sept.p', 2000, 'amt')